<a href="https://colab.research.google.com/github/vplata/Twitter/blob/master/GetTwitterFollowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recopilacion de datos de Twitter

In [0]:
# Import all needed libraries
import tweepy                   # Python wrapper around Twitter API
from google.colab import drive  # to mount Drive to Colab notebook
import json
import csv
from datetime import date
from datetime import datetime
import time


Connect Google Drive

In [0]:
# Connect Google Drive to Colab
drive.mount('/content/gdrive')
# Create a variable to store the data path on your drive
path = './gdrive/My Drive/TwitterStats/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Autenticate to Twitter


In [0]:
# Load Twitter API secrets from an external JSON file
# secrets = json.loads(open(path + 'secrets.json').read())
api_key = 'q1Rg0DBMKvSVTuN6Mg7THOwpA'
api_secret_key = 'Rx02PN0W2a67LcBMIHHJ3PJKOk28OwjRxefVgC9Jvm7M6a7LNl'
access_token = '104981578-AduHmI3pWtWC6atjmr59pF1xYthQcfr1IjnE4ISO'
access_token_secret = 'OdMvlbcV3mmeMP7fzmDpP0bWRxcPNkSizNo7EDDVPhIaY'
# Connect to Twitter API using the secrets
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

Save JSON

In [0]:
def save_json(file_name, file_content):
  with open(path + file_name, 'w', encoding='utf-8') as f:
    json.dump(file_content, f, ensure_ascii=False, indent=4)
    print(path + file_name)

Handle rate limit

In [0]:
# Helper function to handle twitter API rate limit
def limit_handled(cursor, list_name):
  while True:
    try:
      yield cursor.next()
    # Catch Twitter API rate limit exception and wait for 15 minutes
    except tweepy.RateLimitError:
      print("\nData points in list = {}".format(len(list_name)))
      print('Hit Twitter API rate limit.')
      for i in range(3, 0, -1):
        print("Wait for {} mins.".format(i * 5))
        time.sleep(5 * 60)
    # Catch any other Twitter API exceptions
    except tweepy.error.TweepError:
      print('\nCaught TweepError exception' )

Get all twitts

In [0]:
# Helper function to get all tweets of a specified user
# NOTE:This method only allows access to the most recent 3200 tweets
# Source: https://gist.github.com/yanofsky/5436496
def get_all_tweets(screen_name):

  # initialize a list to hold all the Tweets
  alltweets = []
  # make initial request for most recent tweets 
  # (200 is the maximum allowed count)
  new_tweets = api.user_timeline(screen_name = screen_name,count=page_size)
  # save most recent tweets
  alltweets.extend(new_tweets)
  # save the id of the oldest tweet less one to avoid duplication
  oldest = alltweets[-1].id - 1
  # keep grabbing tweets until there are no tweets left
  while len(new_tweets) > 0:
    print("getting tweets before %s" % (oldest))
    # all subsequent requests use the max_id param to prevent
    # duplicates
    new_tweets = api.user_timeline(screen_name = screen_name,count=page_size,max_id=oldest)
    # save most recent tweets
    alltweets.extend(new_tweets)
    # update the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    print("...%s tweets downloaded so far" % (len(alltweets)))
    ### END OF WHILE LOOP ###
  # transform the tweepy tweets into a 2D array that will 
  # populate the csv
  outtweets = [[tweet.id_str, tweet.created_at, tweet.text, tweet.favorite_count,tweet.in_reply_to_screen_name, tweet.retweeted] for tweet in alltweets]
  # write the csv
  with open(path + '%s_tweets.csv' % screen_name, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["id","created_at","text","likes","in reply to","retweeted"])
    writer.writerows(outtweets)
  pass

Followers

In [0]:
# Function to save follower objects in a JSON file.
def get_followers(screen_name):
  
  # Create a list to store follower data
  followers_list = []
  # For-loop to iterate over tweepy cursors
  cursor = tweepy.Cursor(api.followers, screen_name=screen_name, count=page_size).pages(num_pages)
  for i, page in enumerate(limit_handled(cursor, followers_list)):  
    print("\r"+"Loading"+ i % 5 *".", end='')
    
    # Add latest batch of follower data to the list
    followers_list += page
  
  # Extract the follower information
  followers_list = [x._json for x in followers_list]
  # Save the data in a JSON file
  save_json((screen_name+'_followers_data.json'), followers_list)

Following

In [0]:
# Function to save friend objects in a JSON file.
def get_friends(screen_name):
  
  # Create a list to store friends data
  friends_list = []
  # For-loop to iterate over tweepy cursors
  cursor = tweepy.Cursor(api.friends, screen_name=screen_name, count=page_size).pages(num_pages)
  for i, page in enumerate(limit_handled(cursor, friends_list)):  
    print("\r"+"Loading"+ i % 5 *".", end='')
    
    # Add latest batch of friend data to the list
    friends_list += page
  
  # Extract the friends information
  friends_list = [x._json for x in friends_list]
  # Save the data in a JSON file
  save_json((screen_name+'_friends_data.json'), friends_list) 

Todays Stats

In [0]:
# Function to save daily follower and following counts in a JSON file
this_dict = {}

def todays_stats(dict_name, screen_name):
  
  # Get my account information
  #info = api.me()
  info = api.get_user(screen_name)
  # Get follower and following counts
  followers_cnt = info.followers_count  
  following_cnt = info.friends_count
  # Get today's date
  today = date.today()
  d = today.strftime("%b %d, %Y")
  # Save today's stats only if they haven't been collected before
  if d not in dict_name:
    dict_name[d] = {"followers":followers_cnt, "following":following_cnt}
    save_json((screen_name+"_follower_history.json"), dict_name)
  else:
    print('Today\'s stats already exist')

Call Functions

In [0]:
#Set parameters
user = 'txmglobal'
page_size = 200
num_pages = 20

#Call functions
get_all_tweets(user)
get_followers(user)
get_friends(user)
todays_stats(this_dict, user)

getting tweets before 748196030408056831
...171 tweets downloaded so far
Loading./gdrive/My Drive/TwitterStats/txmglobal_followers_data.json


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: generator 'limit_handled' raised StopIteration
  


Loading../gdrive/My Drive/TwitterStats/txmglobal_friends_data.json
./gdrive/My Drive/TwitterStats/txmglobal_follower_history.json
